# Em construção (...)

# Objetivo
Este notebook tem como objetivo demonstrar um passo-a-passo de como capturar dados, fazer análises e ter insights sobre o Open Finance Brasil.
Uso de dados abertos disponíveis no dashboard do cidadão (https://dashboard.openfinancebrasil.org.br/)

# Passo a passo
 - Consumo de APIS do Dashboard do cidadão;
 - Construção de um dataset completo;
 - Análises de dados com gráficos;
 - Insights unindo o conhecimento do Tema com os dados apresentados; 



In [20]:
import pandas as pd
import matplotlib.pyplot as plt
import json
import requests
import urllib3
from datetime import datetime, timedelta
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [16]:
def get_fridays_until_now (year=2023):
    start_date = datetime(year, 1, 1)
    first_friday = start_date + timedelta(days=(4 - start_date.weekday() + 7) % 7)
    fridays = []
    today = datetime.now()

    while first_friday <= today:
        fridays.append(first_friday.strftime('%Y-%m-%d'))
        first_friday += timedelta(days=7)

    return fridays
    
fridays = get_fridays_until_now()

print(fridays)

['2023-01-06', '2023-01-13', '2023-01-20', '2023-01-27', '2023-02-03', '2023-02-10', '2023-02-17', '2023-02-24', '2023-03-03', '2023-03-10', '2023-03-17', '2023-03-24', '2023-03-31', '2023-04-07', '2023-04-14', '2023-04-21', '2023-04-28', '2023-05-05', '2023-05-12', '2023-05-19', '2023-05-26', '2023-06-02', '2023-06-09', '2023-06-16', '2023-06-23', '2023-06-30', '2023-07-07', '2023-07-14', '2023-07-21', '2023-07-28', '2023-08-04', '2023-08-11', '2023-08-18', '2023-08-25', '2023-09-01', '2023-09-08', '2023-09-15', '2023-09-22', '2023-09-29', '2023-10-06', '2023-10-13', '2023-10-20', '2023-10-27', '2023-11-03', '2023-11-10', '2023-11-17', '2023-11-24', '2023-12-01', '2023-12-08', '2023-12-15', '2023-12-22', '2023-12-29', '2024-01-05', '2024-01-12', '2024-01-19', '2024-01-26', '2024-02-02', '2024-02-09', '2024-02-16', '2024-02-23', '2024-03-01', '2024-03-08', '2024-03-15', '2024-03-22', '2024-03-29', '2024-04-05', '2024-04-12', '2024-04-19', '2024-04-26', '2024-05-03', '2024-05-10', '2024

In [11]:
url_opf_participants = "https://data.directory.openbankingbrasil.org.br/participants"
response = requests.get(url_opf_participants)
data_opf_participants = response.json()
banks = {
    item.get("OrganisationName"): item.get("OrganisationId")
    for item in data_opf_participants
}
banks

{'UNIPRIME CENTRAL CCC LTDA.': '18543a69-338d-5e5b-8aa3-dc640ef5f4a2',
 'ITAU CV S.A.': 'a773281f-9897-5e23-b422-ec72735f4f12',
 'EFI S.A. - INSTITUICAO DE PAGAMENTO': 'b43131be-6a7f-5bfb-b436-c3ac5c5c2dac',
 'CARTOS SCD S.A.': 'ea5f58d1-b363-58ef-9d09-46ae53398b5d',
 'NU PAGAMENTOS S.A. - INSTITUICAO DE PAGAMENTO': '926e3037-a685-553c-afa3-f7cb46ff8084',
 'PEFISA S.A. - CFI': 'aaa29038-e09c-5210-afe4-132750a6157c',
 'ACCREDITO - SOCIEDADE DE CREDITO DIRETO S.A.': '12db6790-1ec9-4199-b20a-2820b7ec22c2',
 'DOCK INSTITUICAO DE PAGAMENTO S.A.': 'b20c07c6-ec63-5add-bdef-e1023017f166',
 'SANTANDER CCVM S.A.': '9f1959f0-5fa8-54c7-a978-329994ffe3bd',
 'BCO BRADESCO FINANC. S.A.': '21d26a33-f1bc-5a9e-956c-0bfe048608da',
 'MIDWAY S.A. - SCFI': '42609b73-82bc-5758-9c12-7e0bcc84885e',
 'DLOCAL BRASIL INSTITUICAO DE PAGAMENTO S.A.': '2af3b0a1-b810-4e6d-95f0-95f1a9fd8097',
 'INTRAG DTVM LTDA': '97a95ddb-a6ca-58ff-9551-c371ad20371a',
 'PINBANK BRASIL INSTITUICAO DE PAGAMENTO S/A': 'd0190382-b93b-42a

In [24]:
columns = ["value", "date"]

url = "https://dashboard.openfinancebrasil.org.br/api/consents"
df_actives_consents = pd.DataFrame(columns=columns)
for bank in banks:
    try:
        payload = json.dumps({
            "dates": fridays,
            "bank": banks[bank],
            "role":"client"
        })
        response = requests.post(url, data=payload, verify=False)
        response_data = response.json()
        df_bank = pd.DataFrame(response_data, columns=columns)
        df_bank["bank"] = bank
        df_actives_consents = pd.concat([df_actives_consents, df_bank], ignore_index=True)
    except Exception as e:
        print(f"Error processing bank {bank}: {e}")

df_actives_consents["date"] = pd.to_datetime(df_actives_consents["date"]).dt.date
df_actives_consents["qtt_active_consents"] = pd.to_numeric(df_actives_consents["value"], errors='coerce')
df_actives_consents = df_actives_consents.drop("value",axis=1)


In [25]:
df_actives_consents.head(5)

,date,bank,qtt_active_consents
0,2023-01-06,UNIPRIME CENTRAL CCC LTDA.,21206242
1,2023-01-13,UNIPRIME CENTRAL CCC LTDA.,21910399
2,2023-01-20,UNIPRIME CENTRAL CCC LTDA.,22254725
3,2023-01-27,UNIPRIME CENTRAL CCC LTDA.,23359002
4,2023-02-03,UNIPRIME CENTRAL CCC LTDA.,23879579


In [31]:
columns = ["cpf","cnpj", "date"]
url = "https://dashboard.openfinancebrasil.org.br/api/unique-consents"
df_unique_consents = pd.DataFrame(columns=columns)

for bank in banks:
    try:
        payload = json.dumps({
            "dates": fridays,
            "bank": banks[bank],
            "role":"client"
        })
        response = requests.post(url, data=payload, verify=False)
        response_data = response.json()
        df_bank = pd.DataFrame(response_data, columns=columns)
        df_bank["bank"] = bank
        df_unique_consents = pd.concat([df_unique_consents, df_bank], ignore_index=True)
    except Exception as e:
        print(f"Error processing bank {bank}: {e}")
df_unique_consents["date"] = pd.to_datetime(df_unique_consents["date"]).dt.date
df_unique_consents["qtt_unique_personal_consents"] = pd.to_numeric(df_unique_consents["cpf"], errors='coerce')
df_unique_consents["qtt_unique_business_consents"] = pd.to_numeric(df_unique_consents["cnpj"], errors='coerce')
df_unique_consents = df_unique_consents.drop(["cpf", "cnpj"], axis=1)

In [32]:
df_unique_consents.head(5)

,date,bank,qtt_unique_personal_consents,qtt_unique_business_consents
0,2023-01-06,UNIPRIME CENTRAL CCC LTDA.,14328891,72989
1,2023-01-13,UNIPRIME CENTRAL CCC LTDA.,14559933,75328
2,2023-01-20,UNIPRIME CENTRAL CCC LTDA.,15002147,77614
3,2023-01-27,UNIPRIME CENTRAL CCC LTDA.,15467203,80426
4,2023-02-03,UNIPRIME CENTRAL CCC LTDA.,15861932,83532


In [33]:
df_complete_consents = pd.merge(df_actives_consents, df_unique_consents, on=["date", "bank"], how="outer").fillna(0)
df_complete_consents

,date,bank,qtt_active_consents,qtt_unique_personal_consents,qtt_unique_business_consents
0,2023-01-06,UNIPRIME CENTRAL CCC LTDA.,21206242,14328891,72989
1,2023-01-13,UNIPRIME CENTRAL CCC LTDA.,21910399,14559933,75328
2,2023-01-20,UNIPRIME CENTRAL CCC LTDA.,22254725,15002147,77614
3,2023-01-27,UNIPRIME CENTRAL CCC LTDA.,23359002,15467203,80426
4,2023-02-03,UNIPRIME CENTRAL CCC LTDA.,23879579,15861932,83532
...,...,...,...,...,...
17140,2025-05-09,KLAVI INSTITUICAO DE PAGAMENTOS E GESTAO DE DA...,81027623,52232295,617447
17141,2025-05-16,KLAVI INSTITUICAO DE PAGAMENTOS E GESTAO DE DA...,82405542,53088028,626006
17142,2025-05-23,KLAVI INSTITUICAO DE PAGAMENTOS E GESTAO DE DA...,83972335,54972067,635503
17143,2025-05-30,KLAVI INSTITUICAO DE PAGAMENTOS E GESTAO DE DA...,85553006,55919927,644254
